In [5]:
"""
1. XGBoost Hyper Parameter
2. model 학습 조기 종료 : early_stopping_rounds
3. Best Parameter
"""
from xgboost import XGBClassifier # model
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer # 이항분류
from sklearn.metrics import accuracy_score, classification_report

#################################
## 1. XGBoost Hyper Parameter
#################################

# 1. dataset load
cancer = load_breast_cancer()
print(cancer.feature_names) # 13개 
print()
print(cancer.target_names) # ['malignant' 'benign']

X, y = load_breast_cancer(return_X_y=True)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']

['malignant' 'benign']


In [6]:
# 2. dataset 생성 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
type(X_train) # numpy.ndarray


numpy.ndarray

In [7]:
# 3. model 생성
obj = XGBClassifier()
print(obj) # parameter 확인 
obj = XGBClassifier(colsample_bytree=1,
                    learning_rate=0.1, max_depth=3, 
                    min_child_weight=1,
                    n_estimators=100, 
                    objective="binary:logistic")

# colsample_bytree=1 : 트리를 생성할때 훈련셋에서 feature 샘플링 비율(보통 :0.6~0.9)
# learning_rate=0.1 : 학습율(보통 : 0.01~0.2)
# max_depth=3 : tree 깊이, 과적합 영향
# min_child_weight=1 : 최소 자식 노드 가중치 합(자식 노드 분할 결정), 과적합 영향
# - min_child_weight 보다 더 적은 노드가 생성되면 트리 분할 멈춤
# n_estimators=100 : tree model 수 
# objective='binary:logistic' : 'reg:linear', 'multi:softmax'(num_class 지정)
# [해설] 예측력 향상 : n_estimators는 높이고, learning_rate은 낮춤
#        과적합 영향 : max_depth, min_child_weight  

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)


In [8]:
##############################
## tree model 학습 조기종료 
##############################
evals = [(X_test, y_test)] # 평가 dataset
model = obj.fit(X_train, y_train, eval_metric='error', 
                early_stopping_rounds=50, eval_set=evals, verbose=True)
'''
early_stopping_rounds=100 : 조기종료 파라미터
 - 50개 tree model 학습과정에서 성능평가 지수가 향상되지 않으면 조기종료
eval_metric : tree model 성능평가 방법
 - 이항분류 : error, 멀티클래스 : merror, 회귀 : rmse
'''

[0]	validation_0-error:0.07602
[1]	validation_0-error:0.07018
[2]	validation_0-error:0.07602
[3]	validation_0-error:0.04678
[4]	validation_0-error:0.06433
[5]	validation_0-error:0.05263
[6]	validation_0-error:0.04678
[7]	validation_0-error:0.04094
[8]	validation_0-error:0.04094
[9]	validation_0-error:0.05263
[10]	validation_0-error:0.05263
[11]	validation_0-error:0.05263
[12]	validation_0-error:0.05848
[13]	validation_0-error:0.05263
[14]	validation_0-error:0.05263
[15]	validation_0-error:0.07018
[16]	validation_0-error:0.05848
[17]	validation_0-error:0.05848
[18]	validation_0-error:0.05848
[19]	validation_0-error:0.05263
[20]	validation_0-error:0.05848
[21]	validation_0-error:0.05263
[22]	validation_0-error:0.05263
[23]	validation_0-error:0.05263
[24]	validation_0-error:0.05263
[25]	validation_0-error:0.05263
[26]	validation_0-error:0.05263
[27]	validation_0-error:0.05263
[28]	validation_0-error:0.05263
[29]	validation_0-error:0.05263
[30]	validation_0-error:0.05263
[31]	validation_0-

C:\Users\hyebin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


'\n출력 결과 : 100번 tree model 학습과정에서 26번 이후 성능평가 지수가 최고  \nStopping. Best iteration:\n[26]    validation_0-error:0.005848\n'

In [10]:
# 4. model 평가 
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy =', acc) # accuracy = 0.9549122807017544
print()
report = classification_report(y_test, y_pred)
print(report)

accuracy = 0.9590643274853801

              precision    recall  f1-score   support

           0       0.93      0.97      0.95        65
           1       0.98      0.95      0.97       106

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [13]:

#################################
## 2. Best Parameter
#################################
from sklearn.model_selection import GridSearchCV

# 기본 모델 객체 생성 
obj = XGBClassifier()
                    
params = {'n_estimators':[100, 300],
          'max_depth':[3,  7], 
          'min_child_weight':[1, 3],
          'colsample_bytree':[0.5, 0.7, 1],
          'learning_rate':[0.01, 0.5, 0.1]}


# cv와 iid 생략 : DeprecationWarning 발생(0.24 version에서 삭제 예정) 
gs = GridSearchCV(estimator=obj, param_grid=params, cv=3)
# iid=False : 단순한 폴드 점수의 평균

model = gs.fit(X_train, y_train, eval_metric='error', 
                eval_set=evals, verbose=True) 
# verbose=True : model 테스트 확인 

print('best score =', model.best_score_)
print('best parameter =', model.best_params_)

[0]	validation_0-error:0.08772
[1]	validation_0-error:0.02924
[2]	validation_0-error:0.02924
[3]	validation_0-error:0.03509
[4]	validation_0-error:0.03509
[5]	validation_0-error:0.02924
[6]	validation_0-error:0.04678
[7]	validation_0-error:0.04678
[8]	validation_0-error:0.04094
[9]	validation_0-error:0.02924
[10]	validation_0-error:0.03509
[11]	validation_0-error:0.04094
[12]	validation_0-error:0.04094
[13]	validation_0-error:0.02924
[14]	validation_0-error:0.03509
[15]	validation_0-error:0.03509
[16]	validation_0-error:0.03509
[17]	validation_0-error:0.03509
[18]	validation_0-error:0.03509
[19]	validation_0-error:0.03509
[20]	validation_0-error:0.03509
[21]	validation_0-error:0.03509
[22]	validation_0-error:0.03509
[23]	validation_0-error:0.03509
[24]	validation_0-error:0.03509
[25]	validation_0-error:0.03509
[26]	validation_0-error:0.03509
[27]	validation_0-error:0.03509
[28]	validation_0-error:0.03509
[29]	validation_0-error:0.03509
[30]	validation_0-error:0.03509
[31]	validation_0-

C:\Users\hyebin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[93]	validation_0-error:0.04678
[94]	validation_0-error:0.04678
[95]	validation_0-error:0.05263
[96]	validation_0-error:0.05263
[97]	validation_0-error:0.05263
[98]	validation_0-error:0.04678
[99]	validation_0-error:0.04678
[0]	validation_0-error:0.04678
[1]	validation_0-error:0.06433
[2]	validation_0-error:0.05848
[3]	validation_0-error:0.05263
[4]	validation_0-error:0.05848
[5]	validation_0-error:0.04678
[6]	validation_0-error:0.05848
[7]	validation_0-error:0.05263
[8]	validation_0-error:0.04094
[9]	validation_0-error:0.05263
[10]	validation_0-error:0.05848
[11]	validation_0-error:0.04678
[12]	validation_0-error:0.05263
[13]	validation_0-error:0.05263
[14]	validation_0-error:0.05263
[15]	validation_0-error:0.05263
[16]	validation_0-error:0.05263
[17]	validation_0-error:0.04678
[18]	validation_0-error:0.04678
[19]	validation_0-error:0.04678
[20]	validation_0-error:0.04678
[21]	validation_0-error:0.04678
[22]	validation_0-error:0.04678
[23]	validation_0-error:0.04678
[24]	validation_0-

[149]	validation_0-error:0.02339
[150]	validation_0-error:0.02339
[151]	validation_0-error:0.02339
[152]	validation_0-error:0.02339
[153]	validation_0-error:0.02339
[154]	validation_0-error:0.02924
[155]	validation_0-error:0.02339
[156]	validation_0-error:0.02339
[157]	validation_0-error:0.02339
[158]	validation_0-error:0.02339
[159]	validation_0-error:0.02924
[160]	validation_0-error:0.02924
[161]	validation_0-error:0.02924
[162]	validation_0-error:0.02924
[163]	validation_0-error:0.02924
[164]	validation_0-error:0.02924
[165]	validation_0-error:0.02924
[166]	validation_0-error:0.02924
[167]	validation_0-error:0.02924
[168]	validation_0-error:0.02924
[169]	validation_0-error:0.02924
[170]	validation_0-error:0.02924
[171]	validation_0-error:0.02924
[172]	validation_0-error:0.02924
[173]	validation_0-error:0.02924
[174]	validation_0-error:0.02924
[175]	validation_0-error:0.02924
[176]	validation_0-error:0.02924
[177]	validation_0-error:0.02924
[178]	validation_0-error:0.02924
[179]	vali

[101]	validation_0-error:0.04678
[102]	validation_0-error:0.05263
[103]	validation_0-error:0.05263
[104]	validation_0-error:0.05263
[105]	validation_0-error:0.05263
[106]	validation_0-error:0.05263
[107]	validation_0-error:0.04678
[108]	validation_0-error:0.04678
[109]	validation_0-error:0.04678
[110]	validation_0-error:0.04678
[111]	validation_0-error:0.04678
[112]	validation_0-error:0.04678
[113]	validation_0-error:0.04678
[114]	validation_0-error:0.04678
[115]	validation_0-error:0.04678
[116]	validation_0-error:0.04678
[117]	validation_0-error:0.04678
[118]	validation_0-error:0.04678
[119]	validation_0-error:0.04678
[120]	validation_0-error:0.04678
[121]	validation_0-error:0.04678
[122]	validation_0-error:0.04678
[123]	validation_0-error:0.04678
[124]	validation_0-error:0.04678
[125]	validation_0-error:0.04678
[126]	validation_0-error:0.04678
[127]	validation_0-error:0.04678
[128]	validation_0-error:0.04678
[129]	validation_0-error:0.04678
[130]	validation_0-error:0.04678
[131]	vali

...

[57]	validation_0-error:0.06433
[58]	validation_0-error:0.06433
[59]	validation_0-error:0.06433
[60]	validation_0-error:0.06433
[61]	validation_0-error:0.06433
[62]	validation_0-error:0.06433
[63]	validation_0-error:0.06433
[64]	validation_0-error:0.06433
[65]	validation_0-error:0.06433
[66]	validation_0-error:0.06433
[67]	validation_0-error:0.06433
[68]	validation_0-error:0.06433
[69]	validation_0-error:0.06433
[70]	validation_0-error:0.06433
[71]	validation_0-error:0.06433
[72]	validation_0-error:0.06433
[73]	validation_0-error:0.06433
[74]	validation_0-error:0.06433
[75]	validation_0-error:0.06433
[76]	validation_0-error:0.06433
[77]	validation_0-error:0.06433
[78]	validation_0-error:0.06433
[79]	validation_0-error:0.05848
[80]	validation_0-error:0.06433
[81]	validation_0-error:0.05848
[82]	validation_0-error:0.05848
[83]	validation_0-error:0.05848
[84]	validation_0-error:0.05848
[85]	validation_0-error:0.05848
[86]	validation_0-error:0.05848
[87]	validation_0-error:0.05848
[88]	val

[114]	validation_0-error:0.03509
[115]	validation_0-error:0.03509
[116]	validation_0-error:0.03509
[117]	validation_0-error:0.03509
[118]	validation_0-error:0.03509
[119]	validation_0-error:0.03509
[120]	validation_0-error:0.03509
[121]	validation_0-error:0.03509
[122]	validation_0-error:0.03509
[123]	validation_0-error:0.03509
[124]	validation_0-error:0.03509
[125]	validation_0-error:0.03509
[126]	validation_0-error:0.03509
[127]	validation_0-error:0.03509
[128]	validation_0-error:0.03509
[129]	validation_0-error:0.03509
[130]	validation_0-error:0.03509
[131]	validation_0-error:0.03509
[132]	validation_0-error:0.03509
[133]	validation_0-error:0.02924
[134]	validation_0-error:0.03509
[135]	validation_0-error:0.02924
[136]	validation_0-error:0.03509
[137]	validation_0-error:0.02924
[138]	validation_0-error:0.02924
[139]	validation_0-error:0.02924
[140]	validation_0-error:0.03509
[141]	validation_0-error:0.02924
[142]	validation_0-error:0.02924
[143]	validation_0-error:0.02339
[144]	vali

[65]	validation_0-error:0.06433
[66]	validation_0-error:0.06433
[67]	validation_0-error:0.06433
[68]	validation_0-error:0.06433
[69]	validation_0-error:0.06433
[70]	validation_0-error:0.06433
[71]	validation_0-error:0.06433
[72]	validation_0-error:0.06433
[73]	validation_0-error:0.06433
[74]	validation_0-error:0.06433
[75]	validation_0-error:0.06433
[76]	validation_0-error:0.06433
[77]	validation_0-error:0.06433
[78]	validation_0-error:0.06433
[79]	validation_0-error:0.05848
[80]	validation_0-error:0.06433
[81]	validation_0-error:0.05848
[82]	validation_0-error:0.05848
[83]	validation_0-error:0.05848
[84]	validation_0-error:0.05848
[85]	validation_0-error:0.05848
[86]	validation_0-error:0.05848
[87]	validation_0-error:0.05848
[88]	validation_0-error:0.05848
[89]	validation_0-error:0.05848
[90]	validation_0-error:0.05848
[91]	validation_0-error:0.05848
[92]	validation_0-error:0.05848
[93]	validation_0-error:0.05848
[94]	validation_0-error:0.05848
[95]	validation_0-error:0.05848
[96]	val

[16]	validation_0-error:0.05263
[17]	validation_0-error:0.05848
[18]	validation_0-error:0.05263
[19]	validation_0-error:0.05848
[20]	validation_0-error:0.05848
[21]	validation_0-error:0.05848
[22]	validation_0-error:0.05848
[23]	validation_0-error:0.05848
[24]	validation_0-error:0.05848
[25]	validation_0-error:0.05848
[26]	validation_0-error:0.05848
[27]	validation_0-error:0.05848
[28]	validation_0-error:0.05848
[29]	validation_0-error:0.05848
[30]	validation_0-error:0.05848
[31]	validation_0-error:0.05848
[32]	validation_0-error:0.05848
[33]	validation_0-error:0.05848
[34]	validation_0-error:0.05848
[35]	validation_0-error:0.05263
[36]	validation_0-error:0.05848
[37]	validation_0-error:0.05848
[38]	validation_0-error:0.05848
[39]	validation_0-error:0.05848
[40]	validation_0-error:0.05263
[41]	validation_0-error:0.05848
[42]	validation_0-error:0.05263
[43]	validation_0-error:0.05263
[44]	validation_0-error:0.05263
[45]	validation_0-error:0.05263
[46]	validation_0-error:0.05263
[47]	val

[267]	validation_0-error:0.05263
[268]	validation_0-error:0.05263
[269]	validation_0-error:0.05263
[270]	validation_0-error:0.05263
[271]	validation_0-error:0.05263
[272]	validation_0-error:0.05263
[273]	validation_0-error:0.05263
[274]	validation_0-error:0.05263
[275]	validation_0-error:0.05263
[276]	validation_0-error:0.05263
[277]	validation_0-error:0.05263
[278]	validation_0-error:0.05263
[279]	validation_0-error:0.05263
[280]	validation_0-error:0.05263
[281]	validation_0-error:0.05263
[282]	validation_0-error:0.05263
[283]	validation_0-error:0.05263
[284]	validation_0-error:0.05263
[285]	validation_0-error:0.05263
[286]	validation_0-error:0.05263
[287]	validation_0-error:0.05263
[288]	validation_0-error:0.05263
[289]	validation_0-error:0.05263
[290]	validation_0-error:0.05263
[291]	validation_0-error:0.05263
[292]	validation_0-error:0.05263
[293]	validation_0-error:0.05263
[294]	validation_0-error:0.05263
[295]	validation_0-error:0.05263
[296]	validation_0-error:0.05263
[297]	vali